In [1]:
# Instalar dependencias (ejecutar en notebook si no están)
# !pip install tinydb ipywidgets openai

import os
from tinydb import TinyDB
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display

# --- 1. Inicializar base NoSQL local (TinyDB) ---
db = TinyDB("historial.json")  # se guarda localmente en un archivo JSON

# Inventario simulado aca tendria que ir uan base realacional con todos los productos
inventario = {
    "cerveza artesanal": 10,
    "vino malbec": 5,
    "papas fritas": 20
}

# --- 2. Cliente de OpenAI ---
# asegurate de tener configurada la variable de entorno GEMINI_API_KEY con tu clave de API
genai.configure(api_key="Gemini_API_KEY")

def generar_respuesta(prompt):
    model = genai.GenerativeModel("gemini-2.5-pro") 
    response = model.generate_content(prompt)
    
    return response.text

# --- 3. Lógica de atención al cliente ---
def atender_cliente(consulta):
    # Buscar en inventario
    
    prompt = f"""Quiero que actues con emnpleado de un local que vende diversos productos, siempre vas a respondes 1 o 2 frases cortas y amables elijiendo siempre la opcion mas agradable.
    Primero siempre consutaras tus productos locales que son estos, es importaante que busques el nombre del producto con la mayor similitud {inventario} 
    En el caso negativo donde no se encuentre un producto en el inventario, ofreceras una disculpa y ofreceras pequeña y acotadoa informacion sobre el producto solicitado buscando en internet.
    El cliente pregunta: '{consulta}'. """
    
    respuesta = generar_respuesta(prompt)
    
    # Guardar consulta y respuesta en base local no relacional
    db.insert({"consulta": consulta, "respuesta": respuesta})
    
    return respuesta

# --- 4. Interfaz en notebook ---
entrada = widgets.Text(
    #estaria bueno que se pueda mantener una charla con el historial
    placeholder="Escribí tu consulta como cliente...",
    description="Cliente:",
    layout=widgets.Layout(width="80%")
)

boton = widgets.Button(description="Enviar")
salida = widgets.Output()

def on_submit(_):
    salida.clear_output()
    consulta = entrada.value
    respuesta = atender_cliente(consulta)
    with salida:
        print("Dueño del local:", respuesta)

boton.on_click(on_submit)

display(entrada, boton, salida)


Text(value='', description='Cliente:', layout=Layout(width='80%'), placeholder='Escribí tu consulta como clien…

Button(description='Enviar', style=ButtonStyle())

Output()

InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]